# Google Drive 활용
***

코드를 실행해보기 위해선 다음의 파이썬 라이브러리들이 설치되어있어야 한다.  
- google-api-python-client
- google-auth-oauthlib

Anaconda Prompt에서 다음과 같은 명령어로 설치한다.

```sh
pip install google-api-python-client
pip install google-auth-oauthlib
```

## 1. Google Drive API 사용설정
***


### 1.1. 프로젝트 선택 또는 새 프로젝트 생성
***

[Google API Console](https://console.developers.google.com/)에서 Google Drive API를 사용할 프로젝트를 선택한다.  
프로젝트가 없다면 다음과 같이 새 프로젝트를 만든다.

![새 프로젝트 생성](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/mk_project.gif)


### 1.2. Google Drive API 사용설정
***

![API 사용설정](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/enable_gd_api.gif)


### 1.3. OAuth 동의 화면 설정
***

![OAuth 동의 화면 설정](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/oauth_consent_screen.gif)


### 1.3. 사용자 인증 정보 생성
***

![사용자 인증 정보 생성](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/mk_oauth_key.gif)


### 1.5. 공통적으로 사용하는 코드
***

build()로 리턴되는 drive_service는 <https://developers.google.com/resources/api-libraries/documentation/drive/v3/python/latest>를 참조한다.

In [1]:
# -*- coding: utf-8 -*-
import io
import pprint
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# 다운 받은 키 파일명을 credentials.json로 변경하거나 아래 코드를 변경
CLIENT_SECRET_FILE_NAME = "credentials.json"
TOKEN_FILE_NAME = "token.pickle"

# SCOPES에 들어가는 값들은 다음 URL을 참조: https://developers.google.com/drive/api/v3/about-auth
SCOPES = [
    "https://www.googleapis.com/auth/drive"
]

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists(TOKEN_FILE_NAME):
    with open(TOKEN_FILE_NAME, "rb") as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            CLIENT_SECRET_FILE_NAME, SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open(TOKEN_FILE_NAME, "wb") as token:
        pickle.dump(creds, token)

drive_service = build("drive", "v3", credentials=creds)

## 2. 기본적인 사용
***


### 2.1. 읽기
***


### 2.1.1. 특정 파일/폴더의 ID를 알아내기
***

![파일/폴더의 ID 알아내기](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/get_file_id.gif)



### 2.1.2. 파일/폴더 검색
***

query를 이용하여 원하는 파일 또는 폴더를 검색할 수 있다.  
참조: <https://developers.google.com/drive/api/v3/search-parameters>  
다음은 특정 폴더에 있는 test1.txt라는 파일을 찾는 예제이다.

In [2]:
FOLDER_ID = "1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4"

response = drive_service.files().list(
    q="'{0}' in parents and name = '{1}'".format(FOLDER_ID, "test1.txt"),
    spaces="drive",
    fields="files(id, name, mimeType, size)"
).execute()

files = response.get("files", [])

print("\n##### Metadata #####\n")
print(files)


##### Metadata #####

[{'id': '1Gh1Xdgp9FLFfsf_20Ou_V_mULhz9ZXk5', 'name': 'test1.txt', 'mimeType': 'text/plain', 'size': '3'}]


### 2.1.3. Paging
***
폴더내의 목록을 불러올 때 기본적으로 100개까지 불러오게 되며, pageSize에 값을 줘서 한 번에 조회되는 개수를 조절할 수 있다. (1 ~ 1000개)  
조회하려는 목록이 한 번에 조회되는 목록보다 많을 때는 Paging 처리를 해야 한다.

In [3]:
FOLDER_ID = "17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ"
page_token = None

while True:
    print("\n##### List of Metadata #####\n")

    response = drive_service.files().list(
        q="'{0}' in parents".format(FOLDER_ID),
        spaces="drive",
        fields="nextPageToken, files(id, name, mimeType, parents)",
        pageSize=3,  # 한 번에 조회할 개수
        pageToken=page_token,
        orderBy="name",
    ).execute()

    files = response.get("files", [])
    pprint.pprint(files)

    print("\n##### nextPageToken #####\n")

    page_token = response.get('nextPageToken', None)
    print(page_token)

    if page_token is None:
        break;


##### List of Metadata #####

[{'id': '1hlfZu-LmjCYKduwE7ft8keoEQi_6_t4t',
  'mimeType': 'text/plain',
  'name': 'sub_file_1.txt',
  'parents': ['17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ']},
 {'id': '1gmtqZvJsNku3pbw2XIDFxP2arc_vMaH7',
  'mimeType': 'text/plain',
  'name': 'sub_file_2.txt',
  'parents': ['17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ']},
 {'id': '1Ei3-514r28E1HYAMYxrI7aE9l2adT1hX',
  'mimeType': 'text/plain',
  'name': 'sub_file_3.txt',
  'parents': ['17lrAKEA73zf_EAuimoMmyebVAg-8p2GZ']}]

##### nextPageToken #####

~!!~AI9FV7T59la2smTVqJH4x-lqS0AOLqo5pY4tXG1PeRuKQMceOMFKU4LIxZfkBLIcyfxYMlxFjDReagWwhI_VYeP45ecbwGYk2IK7r7hPl92FXUDDMORy69ThlelKNRDIkB4vP01xcRPT2bZivEjptPvjHiESTHXfzkh5z4BaW3jzP1gafAdvA-hQVOjbuDlw433ZcY4QugsNR3AWvcfuhMX7AOmYjDKacZGXljQHdkj0_jeeSoLwph7JVKA1cLajBX43FYoUqQwXBZRJYdYi5I2hRDY8HLMCoLsDhtRrh9oLSaw-jPjGP3Uk3Nx4VaIm6str8hOshYYBrfTrAaMnO9O7ZdgD__zn-HXT8ZoVl25D1rztlnZr09Y=

##### List of Metadata #####

[{'id': '1BJyvc14JilzVckwU4I6I4IeLaH8FiWgl',
  'mimeType': 'text/pl

### 2.1.4. 폴더/파일의 ID를 이용하여 Metadata를 출력
***

In [4]:
FILE_ID = "1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5"  # 샘플로 읽어볼 파일의 ID
FOLDER_ID = "1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4"  # 샘플로 읽어볼 폴더의 ID

# fields에 들어가는 인자는 다음 URL을 참조: https://developers.google.com/drive/api/v3/reference/files
FIELDS = "kind, id, name, mimeType, size"

print("\n##### File's MetaData #####\n")
response = drive_service.files().get(fileId=FILE_ID, fields=FIELDS).execute()
pprint.pprint(response)

print("\n\n##### Folder's MetaData #####\n")
response = drive_service.files().get(fileId=FOLDER_ID, fields=FIELDS).execute()
pprint.pprint(response)


##### File's MetaData #####

{'id': '1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5',
 'kind': 'drive#file',
 'mimeType': 'text/plain',
 'name': 'Test.txt',
 'size': '10'}


##### Folder's MetaData #####

{'id': '1kBvzgj9MIuDLNqN0cVZhR-wDDfvbKXH4',
 'kind': 'drive#file',
 'mimeType': 'application/vnd.google-apps.folder',
 'name': 'sample1'}


### 2.1.5. Batch Requests
***

In [5]:
FILE_IDS = [
    "1W9tRlK7zu74Iyls2ijHkNkhVo_ftcQBQ",
    "1RN1rbdGfAlZ0tC87Vxib8c89ySgV2xvj",
    "12XmKiypLZnuQKpk_-g8wyu_vvQ0QMQxa",
]

FIELDS = "id, name, mimeType"

def callback(request_id, response, exception):
    if exception:
        # Handle error
        print("\n##### Exception #####\n")
        print(exception)
    else:
        print("\n##### Result #####\n")
        print("Request ID: {0}".format(request_id))
        print("Response: {0}".format(response))

batch = drive_service.new_batch_http_request(callback=callback)
batch.add(drive_service.files().get(fileId=FILE_IDS[0], fields=FIELDS))
batch.add(drive_service.files().get(fileId=FILE_IDS[1], fields=FIELDS))
batch.add(drive_service.files().get(fileId=FILE_IDS[2], fields=FIELDS))
batch.execute()


##### Result #####

Request ID: 1
Response: {'id': '1W9tRlK7zu74Iyls2ijHkNkhVo_ftcQBQ', 'name': 'sub_file2_1.txt', 'mimeType': 'text/plain'}

##### Result #####

Request ID: 2
Response: {'id': '1RN1rbdGfAlZ0tC87Vxib8c89ySgV2xvj', 'name': 'sub_file2_2.txt', 'mimeType': 'text/plain'}

##### Result #####

Request ID: 3
Response: {'id': '12XmKiypLZnuQKpk_-g8wyu_vvQ0QMQxa', 'name': 'sub_file2_3.txt', 'mimeType': 'text/plain'}


### 2.1.6. 파일 다운로드
***

In [6]:
from googleapiclient.http import MediaIoBaseDownload

FILE_ID = "1EFzqg-bfKXOjHP3q7NNUM6QYuEln8sR5"  # 다운로드 받을 파일의 ID

print("\n##### Download Start #####\n")
request = drive_service.files().get_media(fileId=FILE_ID)
buffer = io.BytesIO()
downloader = MediaIoBaseDownload(buffer, request)
done = False

while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

print("\n\n##### Download Result #####\n")
print(buffer.getvalue())


##### Download Start #####

Download 100%.


##### Download Result #####

b'Test Body~'


## 2.2. 쓰기
***

### 2.2.1. 파일 업로드
***

In [7]:
from googleapiclient.http import MediaFileUpload

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

meta_data = {
    "name": "file_up_test.txt",
    "parents": [FOLDER_ID],
}

uploader = MediaFileUpload("upload_test.txt", mimetype="text/plain")
response = drive_service.files().create(body=meta_data, media_body=uploader).execute()

print("\n\n##### Upload Result #####\n")
print(response)



##### Upload Result #####

{'kind': 'drive#file', 'id': '1hqAGzBqbVO7KUq0G_pEttf3lcTUumH2n', 'name': 'file_up_test.txt', 'mimeType': 'text/plain'}


### 2.2.2. 메모리 업로드
***

In [8]:
from googleapiclient.http import MediaIoBaseUpload

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

meta_data = {
    "name": "mem_up_test.txt",
    "parents": [FOLDER_ID],
}

buffer = io.BytesIO()
buffer.write(b"hello world!")

uploader = MediaIoBaseUpload(fd=buffer, mimetype="text/plain")
response = drive_service.files().create(
    body=meta_data, 
    media_body=uploader
).execute()

print("\n\n##### Upload Result #####\n")
print(response)



##### Upload Result #####

{'kind': 'drive#file', 'id': '1OGZdTP_YXn0hpuvatX925YwH47kW97x3', 'name': 'mem_up_test.txt', 'mimeType': 'text/plain'}


## 2.3. 수정
***

In [9]:
FILE_ID = response["id"]  # 위에서 메모리 업로드로 Create된 파일의 ID

meta_data = {
    "name": "mem_up_test2.txt",
}

buffer = io.BytesIO()
buffer.write(b"hi")

uploader = MediaIoBaseUpload(fd=buffer, mimetype="text/plain")
response = drive_service.files().update(
    fileId=FILE_ID,
    body=meta_data, 
    media_body=uploader
).execute()

print("\n##### Upload Result #####\n")
print(response)


##### Upload Result #####

{'kind': 'drive#file', 'id': '1OGZdTP_YXn0hpuvatX925YwH47kW97x3', 'name': 'mem_up_test2.txt', 'mimeType': 'text/plain'}


## 2.4. 삭제
***

In [10]:
response = drive_service.files().delete(fileId=FILE_ID).execute()

print("\n##### Delete Result #####\n")

if 0 == len(response):
    print("Complete")
else:
    print("Failed to delete")


##### Delete Result #####

Complete


# 3. Google Sheets
***


## 3.1. Google Sheets API 사용설정
***

![Google Sheets API 사용설정](https://raw.githubusercontent.com/3x3x3/MediaFiles/master/google_drive/enable_google_sheets.gif)


## 3.2. Sheets Service 객체 생성
***

생성되는 sheets_service는 <https://developers.google.com/resources/api-libraries/documentation/sheets/v4/python/latest/>를 참조한다.

In [11]:
sheets_service = build("sheets", "v4", credentials=creds)

## 3.3. 읽기
***


### 3.3.1. 파일 정보 얻기
***

In [12]:
SPREAD_SHEET_ID = "1PH9fjNRyKd0IqBiaZ3cDGhysmP3Y2241CHCEdt2nQrc"

sheet_obj = sheets_service.spreadsheets()
response = sheet_obj.get(spreadsheetId=SPREAD_SHEET_ID).execute()
pprint.pprint(response)

{'properties': {'autoRecalc': 'ON_CHANGE',
                'defaultFormat': {'backgroundColor': {'blue': 1,
                                                      'green': 1,
                                                      'red': 1},
                                  'padding': {'bottom': 2,
                                              'left': 3,
                                              'right': 3,
                                              'top': 2},
                                  'textFormat': {'bold': False,
                                                 'fontFamily': 'arial,sans,sans-serif',
                                                 'fontSize': 10,
                                                 'foregroundColor': {},
                                                 'italic': False,
                                                 'strikethrough': False,
                                                 'underline': False},
                                

### 3.3.2. 특정 범위 읽기
***

다른 사용법은 <https://developers.google.com/sheets/api/samples/reading>를 참조한다.  
RANGE에 들어갈 A1 Notation은 <https://developers.google.com/sheets/api/guides/concepts#a1_notation>를 참조

In [13]:
SPREAD_SHEET_ID = "1PH9fjNRyKd0IqBiaZ3cDGhysmP3Y2241CHCEdt2nQrc"
SHEET_NAME = response["sheets"][0]["properties"]["title"]
RANGE = "{0}!A1:D3".format(SHEET_NAME)

spreadsheet_obj = sheets_service.spreadsheets()
response = spreadsheet_obj.values().get(spreadsheetId=SPREAD_SHEET_ID, range=RANGE).execute()
pprint.pprint(response)

{'majorDimension': 'ROWS',
 'range': "'시트1'!A1:D3",
 'values': [['', '사과', '딸기', '바나나'],
            ['수량', '2', '10', '3'],
            ['가격', '10000', '5000', '3000']]}


## 3.4. 쓰기
***


### 3.4.1. 특정 폴더에 Spreadsheet 파일을 생성
***

<https://developers.google.com/sheets/api/guides/create>에서 설명한 대로 Spreadsheet 파일을 만들 수도 있지만,  
특정한 폴더내에 파일을 생성하는 방법은 못찾았다.  
Google Drive API를 사용하여 파일을 생성하겠다.  
Spreadsheet의 MIME Type은 "application/vnd.google-apps.spreadsheet"이다,  
구글에서 지원하는 MIME Type는 <https://developers.google.com/drive/api/v3/mime-types>를 참조한다.

In [14]:
import datetime

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 생성
FILE_NAME = "codetbl_{0}".format(datetime.date.today().strftime("%Y%m%d"))

meta_data = {
    "name": FILE_NAME,
    "parents": [FOLDER_ID],
    "mimeType": "application/vnd.google-apps.spreadsheet",
}

response = drive_service.files().create(body=meta_data).execute()

print("\n\n##### Upload Result #####\n")
print(response)



##### Upload Result #####

{'kind': 'drive#file', 'id': '1SAfHB0m6iNukqKnqZK8-uRojA1Qxc7C6h1WzcSwZ_s0', 'name': 'codetbl_20190410', 'mimeType': 'application/vnd.google-apps.spreadsheet'}


### 3.4.2. 생성된 파일에 쓰기
***

다른 사용법은 <https://developers.google.com/sheets/api/samples/writing>를 참조한다.

In [16]:
import pandas as pd

SPREAD_SHEET_ID = response["id"]

df = pd.read_csv("code_info.csv")
print("\n##### Raw Data #####\n")
print(df)

# TODO
# TODO
# TODO
spreadsheet_obj.values().update(spreadsheetId=SPREAD_SHEET_ID, range="시트1!A1:D3")


##### Raw Data #####

       fullcode  shtcode       name
0  KR7005930003  A005930       삼성전자
1  KR7046890000  A046890      서울반도체
2  KR7069500007  A069500  KODEX 200
3  KR7114800006  A114800  KODEX 인버스


In [ ]:
FILE_ID = response["spreadsheetId"]

# 4. 활용
***

## 4.1. 일정한 크기만큼 잘라서 다운로드
***


### 4.1.1. 데이터를 업로드
***

In [ ]:
import struct

FOLDER_ID = "1nWQXXoOGoOIsEJk5GLvUmVqWACWy53Ho"  # 여기에 업로드

df = pd.read_csv("day_005930.csv")
print("\n##### Raw Data #####\n")
print(df)

buffer = io.BytesIO()

for _, row in df.iterrows():
    buffer.write(struct.pack("iiiiii", row["date"], row["open"], row["high"], row["low"], row["close"], row["vol"]))

print("\n##### Binary Data #####\n")
print(buffer.getvalue())

print("\n##### Length of Binary Data #####\n")
print(len(buffer.getvalue()))

meta_data = {
    "name": "day_005930.bry",
    "parents": [FOLDER_ID],
}

uploader = MediaIoBaseUpload(fd=buffer, mimetype="application/octet-stream")
response = drive_service.files().create(
    body=meta_data, 
    media_body=uploader
).execute()

print("\n\n##### Upload Result #####\n")
print(response)

### 4.1.2. 데이터를 다운로드
***

In [ ]:
FILE_ID = response["id"]  # 다운로드 받을 파일의 ID
CHUNK_SIZE = 24


class BufferHandler(io.BytesIO):
    def __init__(self):
        super(BufferHandler, self).__init__()
        self.buffer = None

    def write(self, data):
        self.buffer = data
        super(BufferHandler, self).write(data)

        
print("\n##### Download Start #####\n")
request = drive_service.files().get_media(fileId=FILE_ID)
buffer_handler = BufferHandler()
downloader = MediaIoBaseDownload(buffer_handler, request, chunksize=CHUNK_SIZE - 1)
done = False

while done is False:
    status, done = downloader.next_chunk()
    print(buffer_handler.buffer)
    print(struct.unpack("iiiiii", buffer_handler.buffer))
    print("Download %d%%\n" % int(status.progress() * 100))

print("##### Done #####\n")